## About:
Notebook is used to generate the inputs for the ML model requirements. <br>
The output files for this are:
<ul>
    <li>team_stats.csv - aggregated stats to team level by round</li>
    <li>team_score_results.csv - stacked scored to team level by round</li>
    <li>merged_stat_score_data_clean.csv - combined dataset of aggregated team stats and team scores by round</li>
</ul>

In [4]:
import pandas as pd

In [5]:
# read raw data
game_results = pd.read_csv("data/game_results.csv")
player_stats = pd.read_csv("data/player_stats.csv")

# add season field
player_stats['season'] = [i.strftime('%Y') for i in pd.to_datetime(player_stats['utcStartTime']).to_list()]

C:\Users\SeanW\AppData\Local\Temp\ipykernel_20852\4051413681.py:3: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  player_stats = pd.read_csv("data\player_stats.csv")


In [6]:
# define aggregation columns and grouping levels
cols_to_sum = ['goals', 'behinds', 'kicks', 'handballs', 'disposals', 'marks', 'bounces',
       'tackles', 'contestedPossessions', 'uncontestedPossessions',
       'totalPossessions', 'inside50s', 'marksInside50', 'contestedMarks',
       'hitouts', 'onePercenters', 'disposalEfficiency', 'clangers',
       'freesFor', 'freesAgainst', 'rebound50s',
       'goalAssists', 'turnovers', 'intercepts', 'tacklesInside50', 'shotsAtGoal',
       'scoreInvolvements', 'metresGained', 'clearances.centreClearances',
       'clearances.stoppageClearances', 'clearances.totalClearances',
       'extendedStats.effectiveKicks', 'extendedStats.kickToHandballRatio', 'extendedStats.effectiveDisposals',
       'extendedStats.marksOnLead', 'extendedStats.interceptMarks',
       'extendedStats.hitoutsToAdvantage', 'extendedStats.groundBallGets',
       'extendedStats.f50GroundBallGets', 'extendedStats.scoreLaunches',
       'extendedStats.pressureActs', 'extendedStats.defHalfPressureActs',
       'extendedStats.spoils', 'extendedStats.ruckContests',
       'extendedStats.contestDefOneOnOnes', 'extendedStats.contestDefLosses',
       'extendedStats.contestOffOneOnOnes', 'extendedStats.contestOffWins',
       'extendedStats.centreBounceAttendances', 'extendedStats.kickins',
       'extendedStats.kickinsPlayon']
group_by = ['season','round.roundNumber','team.name']
df_cols = cols_to_sum + group_by

In [7]:
# subset to only aggregation + grouping columns
subset_player_stats = player_stats[df_cols]

In [8]:
# apply aggregation and output file
team_stats = subset_player_stats.groupby(group_by).sum()
team_stats.to_csv('outputs/team_stats.csv')

In [9]:
# get scores & stack the home & away data to the same structure as stats & output to csv
cols = ['year', 'round', 'team', 'score']
home_team = game_results[['round.year','round.roundNumber','match.homeTeam.name','homeTeamScore.matchScore.totalScore']]
home_team.columns = cols
away_team = game_results[['round.year','round.roundNumber','match.awayTeam.name','awayTeamScore.matchScore.totalScore']]
away_team.columns = cols
scores_df = pd.concat([home_team,away_team])
scores_df.reset_index(inplace=True)
scores_df.drop('index', axis=1, inplace=True)
scores_df.to_csv('outputs/team_score_results.csv')

In [10]:
# load processed csvs
team_scores = pd.read_csv('outputs/team_score_results.csv')
team_stats = pd.read_csv('outputs/team_stats.csv')

In [11]:
# merge the datasets
combined_data_raw = team_stats.merge(team_scores, left_on=['season', 'round.roundNumber', 'team.name'], right_on=['year', 'round', 'team'])
combined_data = combined_data_raw.drop(['Unnamed: 0', 'year', 'round', 'team'], axis=1)

In [12]:
# output cleaned data to csv
combined_data.to_csv('outputs/merged_stat_score_data_clean.csv',index=False)